In [ ]:
import gymnasium as gym
from agilerl.components.replay_buffer import ReplayBuffer
from agilerl.algorithms.ddpg import DDPG
from pettingzoo.sisl import multiwalker_v9


# Create environment and Experience Replay Buffer
num_envs = 1
env = multiwalker_v9.parallel_env(render_mode='human')
env.reset()
try:
    state_dim = [env.observation_space(agent).shape[0] for agent in env.agents]
    one_hot = False
    
except Exception:
    state_dim = [env.observation_space(agent).n for agent in env.agents]
    one_hot = True
# Requires one-hot encoding
try:
    action_dim = [env.action_space(agent).shape[0] for agent in env.agents]
except Exception:
    action_dim = [env.action_space(agent).n for agent in env.agents]
    
field_names = ["state", "action", "reward", "next_state", "done"]
memory = ReplayBuffer(memory_size=1000, field_names=field_names, action_dim=action_dim[0])
agent = DDPG(state_dim=state_dim, action_dim=action_dim[0], one_hot=one_hot)   # Create DDPG agent

state = env.reset()[0]  # Reset environment at start of episode
state = state['walker_0']
while True:
    action = agent.getAction(state)[0]    # Get next action from agent
    actions = {agent: action for agent in env.agents}
    next_state, reward, done, _, _ = env.step(actions)   # Act in environment
    for i in done.keys():
        done = [False]
        if i == True :
            done = [True]
            break
    reward = [reward['walker_0']]
    memory.save2memory(state, action, reward, next_state, done)

    # Learn according to learning frequency
    if len(memory) >= agent.batch_size:
        experiences = memory.sample(agent.batch_size) # Sample replay buffer
        agent.learn(experiences)    # Learn according to agent's RL algorithm

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

: 

In [ ]:
import gymnasium as gym
from agilerl.utils.utils import makeVectEnvs
from agilerl.components.replay_buffer import ReplayBuffer
from agilerl.algorithms.ddpg import DDPG

# Create environment and Experience Replay Buffer
num_envs = 1
env = makeVectEnvs('LunarLanderContinuous-v2', num_envs=num_envs)
try:
    state_dim = env.single_observation_space.n          # Discrete observation space
    one_hot = True                                      # Requires one-hot encoding
except:
    state_dim = env.single_observation_space.shape      # Continuous observation space
    one_hot = False                                     # Does not require one-hot encoding
try:
    action_dim = env.single_action_space.n              # Discrete action space
except:
    action_dim = env.single_action_space.shape[0]       # Continuous action space

channels_last = False # Swap image channels dimension from last to first [H, W, C] -> [C, H, W]

if channels_last:
    state_dim = (state_dim[2], state_dim[0], state_dim[1])

field_names = ["state", "action", "reward", "next_state", "done"]
memory = ReplayBuffer(memory_size=10000, field_names=field_names, action_dim=action_dim)

agent = DDPG(state_dim=state_dim, action_dim=action_dim, one_hot=one_hot)   # Create DDPG agent

state = env.reset()[0]  # Reset environment at start of episode
while True:
    if channels_last:
        state = np.moveaxis(state, [-1], [-3])
    action = agent.getAction(state)    # Get next action from agent
    next_state, reward, done, _, _ = env.step(action)   # Act in environment
    print(state)
    print("heh")
    # Save experience to replay buffer
    if channels_last:
        memory.save2memoryVectEnvs(state, action, reward, np.moveaxis(next_state, [-1], [-3]), done)
    else:
        memory.save2memoryVectEnvs(state, action, reward, next_state, done)

    # Learn according to learning frequency
    if len(memory) >= agent.batch_size:
        experiences = memory.sample(agent.batch_size) # Sample replay buffer
        agent.learn(experiences)    # Learn according to agent's RL algorithm

Exception ignored in: <function AsyncVectorEnv.__del__ at 0x7d78c8043d00>
Traceback (most recent call last):
  File "/home/natacha/Documents/myenv/lib/python3.10/site-packages/gymnasium/vector/async_vector_env.py", line 548, in __del__
    self.close(terminate=True)
  File "/home/natacha/Documents/myenv/lib/python3.10/site-packages/gymnasium/vector/vector_env.py", line 271, in close
    self.close_extras(**kwargs)
  File "/home/natacha/Documents/myenv/lib/python3.10/site-packages/gymnasium/vector/async_vector_env.py", line 483, in close_extras
    process.join()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 43, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 27, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 


[[-0.00345974  1.421942   -0.35044846  0.48985133  0.00401577  0.07938185
   0.          0.        ]]
heh
[[-0.00345974  1.421942   -0.35044846  0.48985133  0.00401577  0.07938185
   0.          0.        ]]
heh
[[-0.00345974  1.421942   -0.35044846  0.48985133  0.00401577  0.07938185
   0.          0.        ]]
heh
[[-0.00345974  1.421942   -0.35044846  0.48985133  0.00401577  0.07938185
   0.          0.        ]]
heh
[[-0.00345974  1.421942   -0.35044846  0.48985133  0.00401577  0.07938185
   0.          0.        ]]
heh
[[-0.00345974  1.421942   -0.35044846  0.48985133  0.00401577  0.07938185
   0.          0.        ]]
heh
[[-0.00345974  1.421942   -0.35044846  0.48985133  0.00401577  0.07938185
   0.          0.        ]]
heh
[[-0.00345974  1.421942   -0.35044846  0.48985133  0.00401577  0.07938185
   0.          0.        ]]
heh
[[-0.00345974  1.421942   -0.35044846  0.48985133  0.00401577  0.07938185
   0.          0.        ]]
heh
[[-0.00345974  1.421942   -0.35044846  0.48985